In [1]:
from __future__ import annotations

from qamsi.strategies.scaling.volatility_targeting import VolatilityTargeting
from qamsi.strategies.heuristics.equally_weighted import EWStrategy
from qamsi.cov_estimators.cov_estimators import CovEstimators
from qamsi.utils.data import read_csv
from run import Dataset

REBAL_FREQ = "ME"
TOP_N = 30
DATASET = Dataset.TOPN_US
ESTIMATOR = CovEstimators.RF.value(shrinkage_type="linear")
BASELINE = EWStrategy()

In [2]:
strategy_name = DATASET.name + ESTIMATOR.__class__.__name__ + f"_rebal{REBAL_FREQ}"
baseline_name = DATASET.name + BASELINE.__class__.__name__ + f"_rebal{REBAL_FREQ}"

strategy = read_csv(DATASET.value(topn=TOP_N).SAVE_PATH, strategy_name + ".csv")
baseline = read_csv(DATASET.value(topn=TOP_N).SAVE_PATH, baseline_name + ".csv")

In [3]:
strategy_excess_r = strategy["strategy_xs_r"]
baseline_excess_r = baseline["strategy_xs_r"]
rf = strategy["acc_rate"]

In [4]:
rebal_dates = strategy["rebal"]
rebal_dates = rebal_dates[rebal_dates].index

In [5]:
vol_scaler = VolatilityTargeting(
    trading_lag=1,
    max_exposure=2,
)
scaled_strategy_total_r = vol_scaler(
    strategy_excess_r=strategy_excess_r,
    rebal_dates=rebal_dates,
    baseline=baseline_excess_r,
).add(rf, axis=0)

In [6]:
from qamsi.backtest.assessor import Assessor

assessor = Assessor(
    rf_rate=rf,
    factors=strategy[strategy.columns.difference(["strategy_xs_r", "rebal", "acc_rate"])],
)

In [7]:
assessor(scaled_strategy_total_r)

StrategyStatistics:
* final_nav = 16.949261
* geom_avg_total_r = 0.160509
* geom_avg_xs_r = 0.152730
* std_xs_r = 0.218776
* min_xs_r = -0.125201
* max_xs_r = 0.209715
* skew = 0.543048
* kurtosis = 26.031288
* max_dd = -0.456501
* sharpe = 0.698113
* alpha_buy_hold = 0.082937
* tracking_error_buy_hold = 0.181360
* ir_buy_hold = 0.457308
* factor_loadings = {'low_risk': 0.815182578202651, 'momentum': -0.16725222913071408, 'quality': 0.14273354834939367, 'size': -0.01331071347051014, 'spx': 0.9795376352148033, 'value': -0.33570169229023594}
* alpha_benchmark = 0.067623
* alpha_benchmark_pvalue = 0.037898
* tracking_error_benchmark = 0.148565
* ir_benchmark = 0.455178
* ttest_pval = 0.143841
* levene_pval = 0.847395
* timing_ability_coef = -0.008062
* timing_ability_pval = 0.762108